In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from eval_utils import *
from video_utils import *
from load_utils import *
from tracking_utils import *
from tqdm import tqdm

data_path = '../../AICity_data/train/S03/c010/'

In [2]:
gt_detect = readDetectionsXML('ai_challenge_s03_c010-full_annotation.xml')
gt_notParked = getNotParkedCars(gt_detect)

In [3]:
retinanet_101_detections = {}
with open('./detection_pkls/retinanet_101_detections.pkl','rb') as openFile:
    retinanet_101_detections = pickle.load(openFile)

In [4]:
def remove_overlaps(detections_pd, tolerance=0.9):
    row_to_remove = []
    for detection in detections_pd.get('detection'):
        length, _ = detections_pd.shape
        for i in range(length):
            IoU = detection.IoU(detections_pd.iloc[i]['detection'])
            if IoU > tolerance and IoU < 1:
                row_to_remove.append(i)
                
    row_to_remove = np.unique(np.array(row_to_remove))
    detections_pd = detections_pd.drop(index=row_to_remove)
        
    return detections_pd

In [5]:
def get_detection_dataframe(detections):
    # For detetections in one frame!
    
    bboxes = []
    bsizes = []
    bdetections = []
    tracks = list(range(0, len(detections)))
    
    colours = []
    for i in range(len(detections)):
        colours.append(tuple(np.random.choice(range(256), size=3).astype('int')))
    
    for detection in detections:
        bbox = np.array(detection.getBBox()).astype('int')
        bboxes.append(bbox)

        bsize = int(detection.areaOfRec())
        bsizes.append(bsize)

        bdetections.append(detection)

    detec = {
        'track': tracks,
        'detection': bdetections,
        'bbox': bboxes,
        'size': bsizes,
        'colour': colours,
    }
    detections_pd = pd.DataFrame(detec)
    # detections_pd = detections_pd.sort_values(by=['size'], ascending=False)
    detections_pd = detections_pd.reset_index(drop=True)
    
    return detections_pd

In [6]:
# detections = {}
# with open('tracking_history.pkl','rb') as openFile:
#     detections = pickle.load(openFile)

In [9]:
detections = {}
with open('kalman_tracking.pkl','rb') as openFile:
    detections = pickle.load(openFile)

In [10]:
total_frames = 2141
initial_frame = 535

acc = create_accumulator()

for frame in tqdm(range(initial_frame, total_frames-1)):
    sframe = str(frame)

    gt_detections_pd = get_detection_dataframe(gt_detect[sframe])
    det_detections_pd = detections[frame]
    
    gt_object_ids = []
    gt_objects_boxes = []
    for track in gt_detections_pd['track']:
        gt_object_ids.append(track)
        x = gt_detections_pd[gt_detections_pd['track'] == track].detection.item().xtl
        y = gt_detections_pd[gt_detections_pd['track'] == track].detection.item().ytl
        w = gt_detections_pd[gt_detections_pd['track'] == track].detection.item().w
        h = gt_detections_pd[gt_detections_pd['track'] == track].detection.item().h
        gt_objects_boxes.append([x, y, w, h])

    det_object_ids = []
    det_objects_boxes = []
    for track in det_detections_pd['track']:
        det_object_ids.append(track)
        x = det_detections_pd[det_detections_pd['track'] == track].detection.item().xtl
        y = det_detections_pd[det_detections_pd['track'] == track].detection.item().ytl
        w = det_detections_pd[det_detections_pd['track'] == track].detection.item().w
        h = det_detections_pd[det_detections_pd['track'] == track].detection.item().h
        det_objects_boxes.append([x, y, w, h])

    distances = mm.distances.iou_matrix(gt_objects_boxes, det_objects_boxes, max_iou=1)

    acc, frame_id = update_accumulator(acc, gt_object_ids, det_object_ids, distances)

display_metrics(acc)

100%|██████████| 1605/1605 [01:05<00:00, 24.39it/s]


     num_frames  precision    recall       idp       idr      idf1
acc        1605   0.845693  0.989941  0.749417  0.877243  0.808307
